In [47]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm
import numpy as np

db = sqlite3.connect('../data/hop_teaming.sqlite')


for chunk in tqdm(pd.read_csv('../data/DocGraph_Hop_Teaming_2018.csv', chunksize = 10000)):
    chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]      # Clean up the column names
    chunk.to_sql('hop', db, if_exists = 'append', index = False)            # append to hop table

In [6]:
npi_cols = list(pd.read_csv('../data/npidata_pfile_20050523-20230212.csv', 
                          nrows = 100).columns)

In [7]:
npi_cols

['NPI',
 'Entity Type Code',
 'Replacement NPI',
 'Employer Identification Number (EIN)',
 'Provider Organization Name (Legal Business Name)',
 'Provider Last Name (Legal Name)',
 'Provider First Name',
 'Provider Middle Name',
 'Provider Name Prefix Text',
 'Provider Name Suffix Text',
 'Provider Credential Text',
 'Provider Other Organization Name',
 'Provider Other Organization Name Type Code',
 'Provider Other Last Name',
 'Provider Other First Name',
 'Provider Other Middle Name',
 'Provider Other Name Prefix Text',
 'Provider Other Name Suffix Text',
 'Provider Other Credential Text',
 'Provider Other Last Name Type Code',
 'Provider First Line Business Mailing Address',
 'Provider Second Line Business Mailing Address',
 'Provider Business Mailing Address City Name',
 'Provider Business Mailing Address State Name',
 'Provider Business Mailing Address Postal Code',
 'Provider Business Mailing Address Country Code (If outside U.S.)',
 'Provider Business Mailing Address Telephone Nu

In [67]:
npi_test = pd.read_csv('../data/npidata_pfile_20050523-20230212.csv', 
                       usecols = ['Healthcare Provider Taxonomy Code_1', 'Healthcare Provider Primary Taxonomy Switch_1',
                                  'Healthcare Provider Taxonomy Code_2', 'Healthcare Provider Primary Taxonomy Switch_2',
                                  'Healthcare Provider Taxonomy Code_3', 'Healthcare Provider Primary Taxonomy Switch_3',
                                  'Healthcare Provider Taxonomy Code_4', 'Healthcare Provider Primary Taxonomy Switch_4',
                                  'Healthcare Provider Taxonomy Code_5', 'Healthcare Provider Primary Taxonomy Switch_5',
                                  'Healthcare Provider Taxonomy Code_6', 'Healthcare Provider Primary Taxonomy Switch_6',
                                  'Healthcare Provider Taxonomy Code_7', 'Healthcare Provider Primary Taxonomy Switch_7',
                                  'Healthcare Provider Taxonomy Code_8', 'Healthcare Provider Primary Taxonomy Switch_8',
                                  'Healthcare Provider Taxonomy Code_9', 'Healthcare Provider Primary Taxonomy Switch_9',
                                  'Healthcare Provider Taxonomy Code_10', 'Healthcare Provider Primary Taxonomy Switch_10',
                                  'Healthcare Provider Taxonomy Code_11', 'Healthcare Provider Primary Taxonomy Switch_11',
                                  'Healthcare Provider Taxonomy Code_12', 'Healthcare Provider Primary Taxonomy Switch_12',
                                  'Healthcare Provider Taxonomy Code_13', 'Healthcare Provider Primary Taxonomy Switch_13',
                                  'Healthcare Provider Taxonomy Code_14', 'Healthcare Provider Primary Taxonomy Switch_14',
                                  'Healthcare Provider Taxonomy Code_15', 'Healthcare Provider Primary Taxonomy Switch_15'],
                       skiprows = range(1, 600),  
                       nrows = 1000)
npi_test['primary_taxonomy'] = ''

In [68]:
for index, row in npi_test.iterrows():
    for n in range(1, 16):
        if row['primary_taxonomy'] == '':
            x = str(n)
            if row[f'Healthcare Provider Primary Taxonomy Switch_{x}'] == 'Y':
                npi_test.loc[index, 'primary_taxonomy'] = npi_test.loc[index, f'Healthcare Provider Taxonomy Code_{x}']
                break

npi_test

,Healthcare Provider Taxonomy Code_1,Healthcare Provider Primary Taxonomy Switch_1,Healthcare Provider Taxonomy Code_2,Healthcare Provider Primary Taxonomy Switch_2,Healthcare Provider Taxonomy Code_3,Healthcare Provider Primary Taxonomy Switch_3,Healthcare Provider Taxonomy Code_4,Healthcare Provider Primary Taxonomy Switch_4,Healthcare Provider Taxonomy Code_5,Healthcare Provider Primary Taxonomy Switch_5,...,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Taxonomy Code_12,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Taxonomy Code_13,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Healthcare Provider Primary Taxonomy Switch_15,primary_taxonomy
0,208000000X,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,208000000X
1,1223X0400X,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1223X0400X
2,208000000X,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,208000000X
3,208000000X,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,208000000X
4,208000000X,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,208000000X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,363LF0000X,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,363LF0000X
996,225100000X,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,225100000X
997,207VX0000X,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,207VX0000X
998,183500000X,Y,183500000X,N,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,183500000X


In [63]:
npi_test[['Healthcare Provider Taxonomy Code_1', 'Healthcare Provider Primary Taxonomy Switch_1', 'primary_taxonomy']]

,Healthcare Provider Taxonomy Code_1,Healthcare Provider Primary Taxonomy Switch_1,primary_taxonomy
0,208000000X,Y,208000000X
1,1223X0400X,Y,1223X0400X
2,208000000X,Y,208000000X
3,208000000X,Y,208000000X
4,208000000X,Y,208000000X
...,...,...,...
95,207RP1001X,Y,207RP1001X
96,174400000X,Y,174400000X
97,1223G0001X,Y,1223G0001X
98,363AM0700X,X,


In [66]:
npi_test['Healthcare Provider Primary Taxonomy Switch_1'].value_counts()

Y    958
X     38
N      4
Name: Healthcare Provider Primary Taxonomy Switch_1, dtype: int64

db = sqlite3.connect('../data/npi_data.sqlite')


for chunk in tqdm(pd.read_csv('../data/npidata_pfile_20050523-20230212.csv', 
                              usecols = ['NPI', 'Entity Type Code',],
                              chunksize = 10000)):
    chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]      # Clean up the column names
    chunk.to_sql('hop', db, if_exists = 'append', index = False)            # append to hop table